In [1]:
import json
import pandas as pd
from tqdm.notebook import tqdm
from transformers import MarianTokenizer, MarianMTModel
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# langs = ['de', 'es', 'zh', 'it', 'ro', 'vi', 'ru', 'fr', 'cs', 'jap']
langs = ['fr', 'cs', 'jap']

In [5]:
train.iloc[151]

id                                                      24033
question    What are the military operation which start wi...
category                                             resource
type                                                ['event']
Name: 151, dtype: object

In [2]:
train = pd.read_csv("../../data/at/wikidata/lcquad2_anstype_wikidata_train_translated.csv", sep='$')

In [3]:
train.head()

,id,question,category,type,question_de,question_es,question_zh,question_it,question_ro,question_vi,question_ru,question_fr,question_cs,question_jap
0,19719,What periodical literature does Delta Air Line...,resource,"['publication', 'recurring', 'intellectual wor...",Welche periodische Literatur verwendet Delta A...,¿Qué literatura periódica usa Delta Air Lines ...,三角洲航空公司用什么定期文献作为模具?,Quale letteratura periodica usa Delta Air Line...,Ce literatură periodică foloseşte Delta Air Li...,Không gian thiên văn học dùng để làm đồ trang ...,Какая периодическая литература используется De...,Quelle littérature périodique Delta Air Lines ...,Jakou periodickou literaturu používá Delta Air...,"生れ た 田畑 は, どこ に あ る か. しゃこ を し た 網 の よう に, 祝福..."
1,15554,Who is the child of Ranavalona I's husband?,resource,"['person', 'omnivore', 'natural person']",Wer ist das Kind von Ranavalona I.'s Ehemann?,¿Quién es el hijo del marido de Ranavalona I?,我的丈夫拉纳瓦洛纳的孩子是谁?,Chi è il figlio del marito di Ranavalona I?,Cine e copilul soţului Ranavalonei?,Ai là con trai của Raonan Chồng của em?,Кто ребёнок Ранавалоны я муж?,Qui est l'enfant du mari de Ranavalona?,"Kdo je syn Ranavalony, manžela?",種 を ま く 者 の 子 は だれ か. わたし に は その 夫 の 子 が あ る の...
2,974,Is it true Jeff_Bridges occupation Lane Chandl...,boolean,['boolean'],Ist es wahr Jeff_Bridges Besetzung Lane Chandl...,¿Es verdad la ocupación de Jeff_Bridges Lane C...,这是真的杰夫布里奇占领 莱恩钱德勒和摄影师?,È vero Jeff_Bridges occupazione Lane Chandler ...,E adevărat că Jeff_Bridges se ocupă de Lane Ch...,Có thật là Jeff _Bridges làm việc với Chandler...,"Это правда, что Джефф_Бриджс оккупирует Лейн Ч...",Est-ce vrai Jeff_Bridges occupation Lane Chand...,"Je pravda, Jeff_Bridges povolání Lane Chandler...","これ は 真実 で あ る. すなわち レパイム び と ピセバび と, ザレア び と, ..."
3,27610,Which is the operating income for Qantas?,literal,['number'],Welches ist das operative Ergebnis für Qantas?,¿Cuál es el ingreso operativo de Qantas?,Qantas的营业收入是多少?,Qual è il reddito operativo per Qantas?,Care este venitul din exploatare pentru Qantas?,Thu nhập của Qas là bao nhiêu?,Какой операционный доход для Кантаса?,Quel est le revenu d'exploitation de Qantas?,Jaký je provozní příjem pro Qantas?,"むなし い 争い に よ っ て 人 は, 寒 さ に よ っ て 耕 す こと が でき ..."
4,24488,which cola starts with the letter p,resource,"['soft drink', 'trademark', 'carbonated bevera...",welche Cola mit dem Buchstaben p beginnt,que la cola comienza con la letra p,以字母 p 开头的可口可乐,quale cola inizia con la lettera p,care cola începe cu litera p,mà trong đó khởi khởi khởi khởi khởi động bằng...,Которая начинается с буквы p,qui commence par la lettre p,který Cola začíná písmenem p,彼 は 密接 も し て 神 の み 旨 を 聞 き いれ た.


In [6]:
for lang in langs:
    model = MarianMTModel.from_pretrained(f'Helsinki-NLP/opus-mt-en-{lang}').to(device)
    tokenizer = MarianTokenizer.from_pretrained(f'Helsinki-NLP/opus-mt-en-{lang}')
    translation_list = list()
    
    for text in tqdm(train.question.values):
        # Tokenize the text
        batch = tokenizer([text], return_tensors="pt", padding=True).to(device)

        # Make sure that the tokenized text does not exceed the maximum
        # allowed size of 512
        batch["input_ids"] = batch["input_ids"][:, :512]
        batch["attention_mask"] = batch["attention_mask"][:, :512]
        # Perform the translation and decode the output
        translation = model.generate(**batch)
        translation_list.append(tokenizer.batch_decode(translation, skip_special_tokens=True)[0])
        
    train[f'question_{lang}'] = translation_list
    train.to_csv("../../data/at/wikidata/lcquad2_anstype_wikidata_train_translated.csv", sep='$', index=False)

/home/ins-alex/.local/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
2021-10-16 18:07:17.195568: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


KeyboardInterrupt: 